We have a problem. We don't have normal input for our problem and check something "by hands" is also complicated, because of problem is too hard to get an analytic solutuion. Sorry :(

In [1]:
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
from cvxopt import solvers, matrix

In [2]:
n = 2 # number of points minus 1 of start point
A = np.diag(np.zeros(n) + 1) + np.diag(np.zeros(n - 1) + 1, k=-1)
theta = np.pi / 2 # given theta_1, other theta_i comes our from theta_1

In [3]:
l = np.array([1, 1])
b = np.array([theta, np.pi])
x_start = np.linalg.inv(A) @ b

Our real function to minimize and it's derivative. We drop the first coordinate because $\psi_1$ doesn't defined. Derivative of F in zero we define as a zero.

In [4]:
def F(x):
    x = x[1:]
    L = l[1:]
    return np.array(L * x / np.sin(x))

def dF(x):
    x = x[1:]
    L = l[1:]
    return np.concatenate(([0], L * (np.sin(x) - x * np.cos(x)) / np.sin(x) ** 2))

def hessian(x):
    x = x[1:]
    L = l[1:]
    return np.diag(np.concatenate(([0], L * (1 + 2 * x * np.cos(x) ** 2) / np.sin(x) ** 3)))

Function from cvxopt

In [5]:
def acent(A, b):
    return solvers.cp(F, A=A, b=b)['x']

Methods implementation

In [6]:
def gradient_descent(x_start, eps=1e-4, alpha = 1e-2):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - alpha * dF(x_cur)
    while np.abs(np.linalg.norm(x_cur - x_new)) > eps and np.linalg.norm(dF(x_new)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        if np.min(x_cur) < -np.pi or np.max(x_cur) > np.pi:
            raise Exception('For eps = %f and alpha = %f method comes out from the definition area:('%(eps, alpha))
        x_cur = x_new
        x_new = x_new = x_cur - alpha * dF(x_cur)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('steps: %d' %(step_counter))
    print('x_start =',(x_start))
    print('x_optimal =',(x_cur))

In [7]:
def newton_method(x_start, eps=1, alpha=1):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - np.concatenate(([0], alpha * F(x_cur) / dF(x_cur)[1:]))
    while np.abs(np.linalg.norm(x_cur - x_new)) > eps and np.linalg.norm(F(x_new)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        if np.min(x_cur) < -np.pi or np.max(x_cur) > np.pi:
            raise Exception('For eps = %f and alpha = %f method comes out from the definition area:('%(eps, alpha))
        x_cur = x_new
        x_new = x_cur - np.concatenate(([0], alpha * F(x_cur)[1:] / dF(x_cur)[1:]))
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('steps: %d' %(step_counter))
    print('x_start =',(x_start))
    print('x_optimal =',(x_cur))

In [8]:
gradient_descent(x_start, 1e-10, 1e-1)
newton_method(x_start, 1e-10, 1e-1)

Optimal solution for eps = 0.000000, alpha = 0.100000 founded
steps: 523
x_start = [1.57079633 1.57079633]
x_optimal = [1.57079633e+00 2.26893082e-08]
Optimal solution for eps = 0.000000, alpha = 0.100000 founded
steps: 1
x_start = [1.57079633 1.57079633]
x_optimal = [1.57079633 1.41371669]


In [9]:
def BFGS(F, dF, x_start, eps=10e-10):
    
    step_counter = 0
    I = np.eye(len(x_start))
    H = I
    x_cur = x_start
   
    while np.log(np.linalg.norm(dF(x_cur))) > eps and step_counter < 10000:
        
        p = -H @ dF(x_cur)
        
        # Line search constants for the Wolfe conditions.
        # Repeating the line search
        
        # line_search returns not only alpha
        # but only this value is interesting for us

        alpha = spo.optimize.line_search(F, dF, x_cur, p)[0] 
        
        x_new = x_cur + alpha * p
        s = x_new - x_cur
        y = dF(x_new) - dF(x_cur)
                           
        x_cur = x_new
        
        step_counter += 1
        
        rho = 1.0 / (y.T @ s)
        A1 = I - rho * s[:, np.newaxis] * y[np.newaxis, :]
        A2 = I - rho * y[:, np.newaxis] * s[np.newaxis, :]
        H = np.dot(A1, np.dot(H, A2)) + (rho * s[:, np.newaxis] *
                                                 s[np.newaxis, :])
    
    
    print('Result of BFGS method:')
    print('Final Result (best point):', (x_cur))
    print('Steps: %d' % (step_counter))
    
    #return (x_cur, k)

In [10]:
def F(t):
    x = t[0]
    y = t[1]
    return x ** 2 - x * y + y ** 2 + 9 * x - 6 * y + 20

def dF(t):
    x = t[0]
    y = t[1]
    return np.array([2 * x - y + 9, -x + 2 * y - 6])

In [11]:
x_start = (1, 1)
eps = 1e-10
BFGS(F, dF, x_start)

Result of BFGS method:
Final Result (best point): [-4.10204082  0.87244898]
Steps: 2


On normal input and functions method works.

In [12]:
def F(x):
    return np.cos(x) - x ** 3

def dF(x):
    return -np.sin(x) - 3 * x ** 2

x_start = 0.5

In [13]:
def F(x):
    return (x + 5) ** 2

def dF(x):
    return 2 * (x + 5)

x_start = 3

In [14]:
def test_gradient_descent(x_start, eps=1e-4, alpha = 1e-2):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - alpha * dF(x_cur)
    while np.abs(x_cur - x_new) > eps and np.abs(F(x_new) - F(x_cur)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        x_cur = x_new
        x_new = x_new = x_cur - alpha * dF(x_cur)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('steps: %d' %(step_counter))
    print('x_optimal =',(x_cur))

In [15]:
def test_newton_method(x_start, eps=1e-10, alpha=1):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - F(x_cur) / dF(x_cur)
    while np.abs(np.abs(x_cur - x_new)) > eps and np.linalg.norm(F(x_new)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        x_cur = x_new
        x_new = x_cur - F(x_cur) / dF(x_cur)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('steps: %d' %(step_counter))
    print('x_optimal =',(x_cur))

In [16]:
test_gradient_descent(x_start, 1e-4, 1e-2)
test_newton_method(x_start, 1e-10, 1)

Optimal solution for eps = 0.000100, alpha = 0.010000 founded
steps: 251
x_optimal = -4.949784824391848
Optimal solution for eps = 0.000000, alpha = 1.000000 founded
steps: 19
x_optimal = -4.9999847412109375
